In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500, 'display.max_rows', 500)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
import os, warnings, copy
import matplotlib.pyplot as plt
import sys

sys.path.append('/data/users/morton/git/icrpythonradiomics/machineLearning')
from featureSelection import featureSelection_correlation, featureSelection_groupName


In [2]:
# open clinical spreadsheet
clinicalSpreadsheet = '/Users/morton/Dicom Files/RADSARC_R/ClinicalData/Clinical data for analysis.xlsx'
dfClinical = pd.read_excel(clinicalSpreadsheet, sheet_name='220818_Completed segs', engine='openpyxl')
dfClinical = dfClinical[['Anon Code', 'Grade', 'subtype']]

In [3]:
# open radiomics data
dfRad = pd.read_csv('/Users/morton/Dicom Files/RADSARC_R/XNAT/extractions/extractions__20220910_1006_allRegions/radiomicFeatures/radiomicFeatures.csv')
dfRad.drop(list(dfRad.filter(regex = 'source')), axis = 1, inplace = True)
dfRad.drop(list(dfRad.filter(regex = 'diagnostic')), axis = 1, inplace = True)

# featureSets = ['lesion_original_shape', 'lesion_original_firstorder']
# featureSets = ['lesion_original_firstorder']
# featureSets = ['lesion_original_shape_SurfaceVolumeRatio', 
#                'lesion_original_shape_Sphericity', 
#                'lesion_original_firstorder_Mean',
#               'lesion_original_firstorder_10Percentile']
featureSets = ['lesion_original']

dfRad = dfRad.filter(regex='|'.join(featureSets) + '|StudyPatientName')

dfRad.rename(lambda x:x.replace('lesion_original_',''), axis=1, inplace=True)

# for featureSet in featureSets:
#     dfRad.rename(lambda x:x.replace(featureSet+'_',''), axis=1, inplace=True)

In [4]:
df = dfClinical.merge(dfRad, left_on='Anon Code', right_on='StudyPatientName')
df.drop('Anon Code', axis=1, inplace=True)
df.drop('StudyPatientName', axis=1, inplace=True)
df.drop('Grade', axis=1, inplace=True)
target = 'subtype'

In [5]:
# add pair-wise interactions
# features = list(set(df.columns) - set([target]))
# for i in range(len(features)-1):
#     for j in range(i+1, len(features)):
#         df['interaction_' + features[i] + '_x_' + features[j]] = df[features[i]].multiply(df[features[j]])

In [6]:
df

,subtype,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,shape_MinorAxisLength,shape_Sphericity,shape_SurfaceArea,shape_SurfaceVolumeRatio,shape_VoxelVolume,firstorder_10Percentile,firstorder_90Percentile,firstorder_Energy,firstorder_Entropy,firstorder_InterquartileRange,firstorder_Kurtosis,firstorder_Maximum,firstorder_MeanAbsoluteDeviation,firstorder_Mean,firstorder_Median,firstorder_Minimum,firstorder_Range,firstorder_RobustMeanAbsoluteDeviation,firstorder_RootMeanSquared,firstorder_Skewness,firstorder_TotalEnergy,firstorder_Uniformity,firstorder_Variance,glcm_Autocorrelation,glcm_JointAverage,glcm_ClusterProminence,glcm_ClusterShade,glcm_ClusterTendency,glcm_Contrast,glcm_Correlation,glcm_DifferenceAverage,glcm_DifferenceEntropy,glcm_DifferenceVariance,glcm_JointEnergy,glcm_JointEntropy,glcm_Imc1,glcm_Imc2,glcm_Idm,glcm_Idmn,glcm_Id,glcm_Idn,glcm_InverseVariance,glcm_MaximumProbability,glcm_SumEntropy,glcm_SumSquares,glrlm_GrayLevelNonUniformity,glrlm_GrayLevelNonUniformityNormalized,glrlm_GrayLevelVariance,glrlm_HighGrayLevelRunEmphasis,glrlm_LongRunEmphasis,glrlm_LongRunHighGrayLevelEmphasis,glrlm_LongRunLowGrayLevelEmphasis,glrlm_LowGrayLevelRunEmphasis,glrlm_RunEntropy,glrlm_RunLengthNonUniformity,glrlm_RunLengthNonUniformityNormalized,glrlm_RunPercentage,glrlm_RunVariance,glrlm_ShortRunEmphasis,glrlm_ShortRunHighGrayLevelEmphasis,glrlm_ShortRunLowGrayLevelEmphasis,glszm_GrayLevelNonUniformity,glszm_GrayLevelNonUniformityNormalized,glszm_GrayLevelVariance,glszm_HighGrayLevelZoneEmphasis,glszm_LargeAreaEmphasis,glszm_LargeAreaHighGrayLevelEmphasis,glszm_LargeAreaLowGrayLevelEmphasis,glszm_LowGrayLevelZoneEmphasis,glszm_SizeZoneNonUniformity,glszm_SizeZoneNonUniformityNormalized,glszm_SmallAreaEmphasis,glszm_SmallAreaHighGrayLevelEmphasis,glszm_SmallAreaLowGrayLevelEmphasis,glszm_ZoneEntropy,glszm_ZonePercentage,glszm_ZoneVariance,gldm_DependenceEntropy,gldm_DependenceNonUniformity,gldm_DependenceNonUniformityNormalized,gldm_DependenceVariance,gldm_GrayLevelNonUniformity,gldm_GrayLevelVariance,gldm_HighGrayLevelEmphasis,gldm_LargeDependenceEmphasis,gldm_LargeDependenceHighGrayLevelEmphasis,gldm_LargeDependenceLowGrayLevelEmphasis,gldm_LowGrayLevelEmphasis,gldm_SmallDependenceEmphasis,gldm_SmallDependenceHighGrayLevelEmphasis,gldm_SmallDependenceLowGrayLevelEmphasis,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Complexity,ngtdm_Contrast,ngtdm_Strength,histogram_5Percentile,histogram_15Percentile,histogram_20Percentile,histogram_25Percentile,histogram_30Percentile,histogram_35Percentile,histogram_40Percentile,histogram_45Percentile,histogram_55Percentile,histogram_60Percentile,histogram_65Percentile,histogram_70Percentile,histogram_75Percentile,histogram_80Percentile,histogram_85Percentile,histogram_95Percentile
0,LMS,0.832921,0.798058,50.502192,63.281392,72.945185,72.532751,64.257295,75.099933,1.165954e+05,52.708409,0.757343,15239.796482,0.130707,116855.0,4.681213,58.148197,3.614182e+07,1.941701,27.260132,5.443797,106.783615,17.366682,31.794965,33.647402,-125.144635,231.928249,11.371108,39.324787,-0.879974,1.807091e+08,0.320175,535.519050,61.405001,7.812533,31.676884,-2.984671,2.436771,0.959456,0.434987,0.701928,1.430707,0.466753,0.119122,3.631130,-0.076343,0.500399,0.674708,0.992265,0.688946,0.943123,0.504461,0.243837,2.628258,0.849057,8.055529e+04,0.274470,1.175761,60.717691,2.801018,173.914765,0.047520,0.018233,3.285204,1.635333e+05,0.557195,0.697669,0.746542,0.775021,46.649809,0.014521,1172.948680,0.219694,2.231184,57.885746,266.227571,1.694554e+04,4.206300,0.021081,1778.477056,0.333111,0.598540,33.436616,0.013806,4.772793,0.228446,247.065839,4.365159,3731.801506,0.159677,3.035793,7.482814e+03,0.939881,61.318985,15.892302,996.340850,0.260451,0.017668,0.213099,12.351136,0.004470,29.030057,0.000273,41.397643,0.011702,0.019347,-12.0,7.0,12.0,16.0,20.0,24.0,27.0,30.0,37.0,40.0,

In [13]:
random_state = 42
np.random.seed(random_state)

X = df.drop(target, axis=1)
y = df[target]

correlationHierarchy = ['shape_MeshVolume', 'shape', 'firstorder']
fsc = featureSelection_correlation(threshold=0.9, exact=False, featureGroupHierarchy=correlationHierarchy)

model = LogisticRegressionCV(penalty='l1', multi_class='multinomial', solver='saga', Cs=20, cv=5, random_state=random_state, max_iter=50000)

pipe = Pipeline([('fsc', fsc),
                 ('scaler', StandardScaler()), 
                 ('classifier', model)])

In [14]:
pipe.fit(X,y)
coef = pd.DataFrame({'Feature':list(np.array(X.columns)[pipe.steps[0][1].mask_]), 'Coef':list(pipe.steps[2][1].coef_[0])})
coef = coef.sort_values(by='Coef', ascending=False, key=abs)
coef = coef.loc[coef.Coef != 0,:]
coef

,Feature,Coef
4,firstorder_90Percentile,-0.690006
16,glrlm_RunEntropy,0.595666
14,glcm_InverseVariance,0.191322
3,shape_Sphericity,-0.136674
2,shape_MeshVolume,0.079317
0,shape_Elongation,-0.070658
10,firstorder_RootMeanSquared,-0.061169
7,firstorder_Maximum,0.047391
20,glszm_SmallAreaEmphasis,0.046037


In [ ]:
n_splits = 10
n_repeats = 1
validation = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats)

# supress warnings for cross_validate
warnings.simplefilter("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"
#
cv = cross_validate(pipe, X, y, cv=validation, scoring='roc_auc', n_jobs=-1)
#
warnings.simplefilter('default')
os.environ["PYTHONWARNINGS"] = 'default'

print('AUROC (CV) = ' + str(np.round(np.mean(cv['test_score']),5)))